# Implementacija stručnog sustava za životinje

Primjer iz [AI for Beginners Curriculum](http://github.com/microsoft/ai-for-beginners).

U ovom primjeru implementirat ćemo jednostavan sustav temeljen na znanju za određivanje životinje na temelju nekih fizičkih karakteristika. Sustav se može prikazati sljedećim AND-OR stablom (ovo je dio cijelog stabla, lako možemo dodati još pravila):

![](../../../../translated_images/AND-OR-Tree.5592d2c70187f283703c8e9c0d69d6a786eb370f4ace67f9a7aae5ada3d260b0.hr.png)


## Naša vlastita ljuska za ekspertne sustave s unatrag zaključivanjem

Pokušajmo definirati jednostavan jezik za reprezentaciju znanja temeljen na produkcijskim pravilima. Koristit ćemo Python klase kao ključne riječi za definiranje pravila. Postojat će tri osnovne vrste klasa:
* `Ask` predstavlja pitanje koje treba postaviti korisniku. Sadrži skup mogućih odgovora.
* `If` predstavlja pravilo i samo je sintaktički šećer za pohranu sadržaja pravila.
* `AND`/`OR` su klase za predstavljanje AND/OR grana stabla. One samo pohranjuju popis argumenata unutar sebe. Kako bismo pojednostavili kod, sva funkcionalnost je definirana u roditeljskoj klasi `Content`.


In [1]:
class Ask():
    def __init__(self,choices=['y','n']):
        self.choices = choices
    def ask(self):
        if max([len(x) for x in self.choices])>1:
            for i,x in enumerate(self.choices):
                print("{0}. {1}".format(i,x),flush=True)
            x = int(input())
            return self.choices[x]
        else:
            print("/".join(self.choices),flush=True)
            return input()

class Content():
    def __init__(self,x):
        self.x=x
        
class If(Content):
    pass

class AND(Content):
    pass

class OR(Content):
    pass

U našem sustavu, radna memorija bi sadržavala popis **činjenica** kao **parova atribut-vrijednost**. Baza znanja može se definirati kao jedan veliki rječnik koji povezuje radnje (nove činjenice koje bi trebale biti umetnute u radnu memoriju) s uvjetima, izraženima kao AND-OR izrazi. Također, neke činjenice mogu biti `Pitaj`-ane.


In [2]:
rules = {
    'default': Ask(['y','n']),
    'color' : Ask(['red-brown','black and white','other']),
    'pattern' : Ask(['dark stripes','dark spots']),
    'mammal': If(OR(['hair','gives milk'])),
    'carnivor': If(OR([AND(['sharp teeth','claws','forward-looking eyes']),'eats meat'])),
    'ungulate': If(['mammal',OR(['has hooves','chews cud'])]),
    'bird': If(OR(['feathers',AND(['flies','lies eggs'])])),
    'animal:monkey' : If(['mammal','carnivor','color:red-brown','pattern:dark spots']),
    'animal:tiger' : If(['mammal','carnivor','color:red-brown','pattern:dark stripes']),
    'animal:giraffe' : If(['ungulate','long neck','long legs','pattern:dark spots']),
    'animal:zebra' : If(['ungulate','pattern:dark stripes']),
    'animal:ostrich' : If(['bird','long nech','color:black and white','cannot fly']),
    'animal:pinguin' : If(['bird','swims','color:black and white','cannot fly']),
    'animal:albatross' : If(['bird','flies well'])
}

Za izvođenje unatrag zaključivanja, definirat ćemo klasu `Knowledgebase`. Ona će sadržavati:
* Radnu `memoriju` - rječnik koji mapira atribute na vrijednosti
* Pravila `Knowledgebase` u formatu definiranom gore

Dvije glavne metode su:
* `get` za dobivanje vrijednosti atributa, uz izvođenje zaključivanja ako je potrebno. Na primjer, `get('color')` bi dobio vrijednost za atribut boje (pitati će ako je potrebno i pohraniti vrijednost za kasniju upotrebu u radnoj memoriji). Ako pitamo `get('color:blue')`, tražit će boju, a zatim vratiti vrijednost `y`/`n` ovisno o boji.
* `eval` izvodi stvarno zaključivanje, tj. prolazi kroz AND/OR stablo, procjenjuje podciljeve itd.


In [3]:
class KnowledgeBase():
    def __init__(self,rules):
        self.rules = rules
        self.memory = {}
        
    def get(self,name):
        if ':' in name:
            k,v = name.split(':')
            vv = self.get(k)
            return 'y' if v==vv else 'n'
        if name in self.memory.keys():
            return self.memory[name]
        for fld in self.rules.keys():
            if fld==name or fld.startswith(name+":"):
                # print(" + proving {}".format(fld))
                value = 'y' if fld==name else fld.split(':')[1]
                res = self.eval(self.rules[fld],field=name)
                if res!='y' and res!='n' and value=='y':
                    self.memory[name] = res
                    return res
                if res=='y':
                    self.memory[name] = value
                    return value
        # field is not found, using default
        res = self.eval(self.rules['default'],field=name)
        self.memory[name]=res
        return res
                
    def eval(self,expr,field=None):
        # print(" + eval {}".format(expr))
        if isinstance(expr,Ask):
            print(field)
            return expr.ask()
        elif isinstance(expr,If):
            return self.eval(expr.x)
        elif isinstance(expr,AND) or isinstance(expr,list):
            expr = expr.x if isinstance(expr,AND) else expr
            for x in expr:
                if self.eval(x)=='n':
                    return 'n'
            return 'y'
        elif isinstance(expr,OR):
            for x in expr.x:
                if self.eval(x)=='y':
                    return 'y'
            return 'n'
        elif isinstance(expr,str):
            return self.get(expr)
        else:
            print("Unknown expr: {}".format(expr))

Sada definirajmo našu bazu znanja o životinjama i obavimo konzultaciju. Imajte na umu da će vas ovaj poziv postavljati pitanja. Možete odgovoriti upisivanjem `y`/`n` za pitanja s odgovorima da-ne, ili navođenjem broja (0..N) za pitanja s duljim odgovorima s više izbora.


In [4]:
kb = KnowledgeBase(rules)
kb.get('animal')

hair
y/n
sharp teeth
y/n
claws
y/n
forward-looking eyes
y/n
color
0. red-brown
1. black and white
2. other
has hooves
y/n
long neck
y/n
long legs
y/n
pattern
0. dark stripes
1. dark spots


'giraffe'

## Korištenje PyKnow za naprijedno zaključivanje

U sljedećem primjeru pokušat ćemo implementirati naprijedno zaključivanje koristeći jednu od biblioteka za reprezentaciju znanja, [PyKnow](https://github.com/buguroo/pyknow/). **PyKnow** je biblioteka za kreiranje sustava naprijednog zaključivanja u Pythonu, koja je dizajnirana da bude slična klasičnom starom sustavu [CLIPS](http://www.clipsrules.net/index.html).

Također smo mogli sami implementirati naprijedno povezivanje bez većih problema, ali naivne implementacije obično nisu vrlo učinkovite. Za učinkovitije podudaranje pravila koristi se poseban algoritam [Rete](https://en.wikipedia.org/wiki/Rete_algorithm).


In [5]:
import sys
!{sys.executable} -m pip install git+https://github.com/buguroo/pyknow/

  Cloning https://github.com/buguroo/pyknow/ to /tmp/pip-req-build-3cqeulyl
  Running command git clone --filter=blob:none --quiet https://github.com/buguroo/pyknow/ /tmp/pip-req-build-3cqeulyl
  Resolved https://github.com/buguroo/pyknow/ to commit 48818336f2e9a126f1964f2d8dc22d37ff800fe8
  Preparing metadata (setup.py) ... done
  Using cached frozendict-1.2.tar.gz (2.6 kB)
  Preparing metadata (setup.py) ... done
  Using cached schema-0.6.7-py2.py3-none-any.whl (14 kB)
  Created wheel for pyknow: filename=pyknow-1.7.0-py3-none-any.whl size=34228 sha256=b7de5b09292c4007667c72f69b98d5a1b5f7324ff15f9dd8e077c3d5f7aade42
  Stored in directory: /tmp/pip-ephem-wheel-cache-k7jpave7/wheels/81/1a/d3/f6c15dbe1955598a37755215f2a10449e7418500d7bd4b9508
  Created wheel for frozendict: filename=frozendict-1.2-py3-none-any.whl size=3148 sha256=2863d55c240d2409cddf05ccfe600591f8478681549fc97555c47c90dc6bb160
  Stored in directory: /home/rg/.cache/pip/wheels/49/ac/f8/cb8120244e710bdb479c86198b03c7b08c

In [13]:
from pyknow import *
#import pyknow

Definirat ćemo naš sustav kao klasu koja nasljeđuje `KnowledgeEngine`. Svako pravilo definirano je zasebnom funkcijom s anotacijom `@Rule`, koja specificira kada bi se pravilo trebalo aktivirati. Unutar pravila možemo dodavati nove činjenice koristeći funkciju `declare`, a dodavanje tih činjenica rezultirat će pozivanjem dodatnih pravila od strane stroja za zaključivanje unaprijed.


In [14]:
class Animals(KnowledgeEngine):
    @Rule(OR(
           AND(Fact('sharp teeth'),Fact('claws'),Fact('forward looking eyes')),
           Fact('eats meat')))
    def cornivor(self):
        self.declare(Fact('carnivor'))
        
    @Rule(OR(Fact('hair'),Fact('gives milk')))
    def mammal(self):
        self.declare(Fact('mammal'))

    @Rule(Fact('mammal'),
          OR(Fact('has hooves'),Fact('chews cud')))
    def hooves(self):
        self.declare('ungulate')
        
    @Rule(OR(Fact('feathers'),AND(Fact('flies'),Fact('lays eggs'))))
    def bird(self):
        self.declare('bird')
        
    @Rule(Fact('mammal'),Fact('carnivor'),
          Fact(color='red-brown'),
          Fact(pattern='dark spots'))
    def monkey(self):
        self.declare(Fact(animal='monkey'))

    @Rule(Fact('mammal'),Fact('carnivor'),
          Fact(color='red-brown'),
          Fact(pattern='dark stripes'))
    def tiger(self):
        self.declare(Fact(animal='tiger'))

    @Rule(Fact('ungulate'),
          Fact('long neck'),
          Fact('long legs'),
          Fact(pattern='dark spots'))
    def giraffe(self):
        self.declare(Fact(animal='giraffe'))

    @Rule(Fact('ungulate'),
          Fact(pattern='dark stripes'))
    def zebra(self):
        self.declare(Fact(animal='zebra'))

    @Rule(Fact('bird'),
          Fact('long neck'),
          Fact('cannot fly'),
          Fact(color='black and white'))
    def straus(self):
        self.declare(Fact(animal='ostrich'))

    @Rule(Fact('bird'),
          Fact('swims'),
          Fact('cannot fly'),
          Fact(color='black and white'))
    def pinguin(self):
        self.declare(Fact(animal='pinguin'))

    @Rule(Fact('bird'),
          Fact('flies well'))
    def albatros(self):
        self.declare(Fact(animal='albatross'))
        
    @Rule(Fact(animal=MATCH.a))
    def print_result(self,a):
          print('Animal is {}'.format(a))
                    
    def factz(self,l):
        for x in l:
            self.declare(x)

Nakon što definiramo bazu znanja, popunjavamo radnu memoriju s nekim početnim činjenicama, a zatim pozivamo metodu `run()` za izvođenje zaključivanja. Kao rezultat možete vidjeti da se nove izvedene činjenice dodaju u radnu memoriju, uključujući konačnu činjenicu o životinji (ako smo ispravno postavili sve početne činjenice).


In [15]:
ex1 = Animals()
ex1.reset()
ex1.factz([
    Fact(color='red-brown'),
    Fact(pattern='dark stripes'),
    Fact('sharp teeth'),
    Fact('claws'),
    Fact('forward looking eyes'),
    Fact('gives milk')])
ex1.run()
ex1.facts

Animal is tiger


FactList([(0, InitialFact()),
          (1, Fact(color='red-brown')),
          (2, Fact(pattern='dark stripes')),
          (3, Fact('sharp teeth')),
          (4, Fact('claws')),
          (5, Fact('forward looking eyes')),
          (6, Fact('gives milk')),
          (7, Fact('mammal')),
          (8, Fact('carnivor')),
          (9, Fact(animal='tiger'))])


---

**Odricanje od odgovornosti**:  
Ovaj dokument je preveden korištenjem AI usluge za prevođenje [Co-op Translator](https://github.com/Azure/co-op-translator). Iako nastojimo osigurati točnost, imajte na umu da automatski prijevodi mogu sadržavati pogreške ili netočnosti. Izvorni dokument na izvornom jeziku treba smatrati mjerodavnim izvorom. Za ključne informacije preporučuje se profesionalni prijevod od strane stručnjaka. Ne preuzimamo odgovornost za bilo kakva nesporazuma ili pogrešna tumačenja koja mogu proizaći iz korištenja ovog prijevoda.
